In [1]:
import numpy as np
import math

# Import Qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi


def alice_bob_circuit(a,b):
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0,1)
    if a==1:
        qc.h(0)
    if b==0:
        qc.ry(-math.pi/4.0,1)
    else:
        qc.ry(math.pi/4.0,1)
    qc.measure_all()
    return qc

def alice_bob_measurements(simulator,verbose=True):
    counts = {}
    for a in range(2):
        for b in range(2):
            qc = alice_bob_circuit(a,b)
            if verbose:
                print(f'\nAlice: {a}, Bob: {b}')
                print(qc.draw(output='text'))
            qc = transpile(qc, simulator)
            result = simulator.run(qc).result()
            counts[(a,b)] = result.get_counts(qc)
            if verbose:
                print(counts[a,b])
    return counts

# Transpile for simulator
simulator = Aer.get_backend('aer_simulator')
counts = alice_bob_measurements(simulator)


Alice: 0, Bob: 0
        ┌───┐                  ░ ┌─┐   
   q_0: ┤ H ├──■───────────────░─┤M├───
        └───┘┌─┴─┐┌──────────┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(-π/4) ├─░──╫─┤M├
             └───┘└──────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════╩══╩═
                                  0  1 
{'11': 436, '00': 443, '01': 56, '10': 89}

Alice: 0, Bob: 1
        ┌───┐                 ░ ┌─┐   
   q_0: ┤ H ├──■──────────────░─┤M├───
        └───┘┌─┴─┐┌─────────┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(π/4) ├─░──╫─┤M├
             └───┘└─────────┘ ░  ║ └╥┘
meas: 2/═════════════════════════╩══╩═
                                 0  1 
{'00': 460, '11': 423, '10': 69, '01': 72}

Alice: 1, Bob: 0
        ┌───┐        ┌───┐     ░ ┌─┐   
   q_0: ┤ H ├──■─────┤ H ├─────░─┤M├───
        └───┘┌─┴─┐┌──┴───┴───┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(-π/4) ├─░──╫─┤M├
             └───┘└──────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════╩══╩═
                                  0  1 
{'00': 439, '11': 427, '01'

In [2]:
# We now go ahead and assign red and green to the measurement 
# outcomes (said as 0 and 1), and score the game.

def scoring(coins,counts):
        win = lose = 0 
        if coins[0]*coins[1]==1:
            # head-heads
            for k in counts:
                # win on different colors
                if k=='10' or k=='01':
                    win += counts[k]
                else:
                    lose += counts[k]
        else:
            # not head-heads
            for k in counts:
                # win on same colors
                if k=='00' or k=='11':
                    win += counts[k]
                else:
                    lose += counts[k]
        return(win,lose)
    
def alice_bob_scoring(counts):
    
    def make_into_colors(k):
        color = {'0':'G','1':'R'}
        return ''.join([color[k[0]],'',color[k[1]]])

    overall = 0.0
    for a in range(2):
        for b in range(2):
            print(f'\nAlice: {a}, Bob: {b}')
            c = counts[(a,b)]
            score = scoring((a,b),c)
            for k in c:
                print(f'\t{make_into_colors(k)}: {c[k]}')
            print(f'\twin: {score[0]}, lose: {score[1]}')
            w = score[0]/(score[0]+score[1])
            overall += w 
            print(f'\tadvantage: {w:.4f}')
    return overall/4.0
                
w = alice_bob_scoring(counts)
print(f'\nGame advantage: {w:.4f}')



Alice: 0, Bob: 0
	RR: 436
	GG: 443
	GR: 56
	RG: 89
	win: 879, lose: 145
	advantage: 0.8584

Alice: 0, Bob: 1
	GG: 460
	RR: 423
	RG: 69
	GR: 72
	win: 883, lose: 141
	advantage: 0.8623

Alice: 1, Bob: 0
	GG: 439
	RR: 427
	GR: 74
	RG: 84
	win: 866, lose: 158
	advantage: 0.8457

Alice: 1, Bob: 1
	RG: 416
	GR: 450
	GG: 80
	RR: 78
	win: 866, lose: 158
	advantage: 0.8457

Game advantage: 0.8530


In [5]:
# repeat the preliminaries
# repeating the code from above


import numpy as np
import math

# Import Qiskit
from qiskit import QuantumCircuit
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi

simulator = Aer.get_backend('aer_simulator')

def alice_bob_measurements(simulator,verbose=True):
    counts = {}
    for a in range(2):
        for b in range(2):
            qc = alice_bob_circuit(a,b)
            if verbose:
                print(f'\nAlice: {a}, Bob: {b}')
                print(qc.draw(output='text'))
            qc = transpile(qc, simulator)
            result = simulator.run(qc).result()
            counts[(a,b)] = result.get_counts(qc)
            if verbose:
                print(counts[a,b])
    return counts

def scoring(coins,counts):
        win = lose = 0 
        if coins[0]*coins[1]==1:
            # head-heads
            for k in counts:
                # win on different colors
                if k=='10' or k=='01':
                    win += counts[k]
                else:
                    lose += counts[k]
        else:
            # not head-heads
            for k in counts:
                # win on same colors
                if k=='00' or k=='11':
                    win += counts[k]
                else:
                    lose += counts[k]
        return(win,lose)
    
def alice_bob_scoring(counts):
    
    def make_into_colors(k):
        color = {'0':'G','1':'R'}
        return ''.join([color[k[0]],'',color[k[1]]])

    overall = 0.0
    for a in range(2):
        for b in range(2):
            print(f'\nAlice: {a}, Bob: {b}')
            c = counts[(a,b)]
            score = scoring((a,b),c)
            for k in c:
                print(f'\t{make_into_colors(k)}: {c[k]}')
            print(f'\twin: {score[0]}, lose: {score[1]}')
            w = score[0]/(score[0]+score[1])
            overall += w 
            print(f'\tadvantage: {w:.4f}')
    return overall/4.0


In [11]:
def create_bell_pair():
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0,1)
    return qc

def measure_at_theta(qc,theta):
    qc.ry(theta,0)
    qc.ry(theta,1)
    qc.measure_all()
    return qc

for i in range(13):
    theta = math.pi/12*i
    qc = measure_at_theta(create_bell_pair(),theta)
    print(qc.draw(output='text'))
    qc = transpile(qc, simulator)
    result = simulator.run(qc).result()
    print(result.get_counts(qc))
    
simulator = Aer.get_backend('aer_simulator')
counts = alice_bob_measurements(simulator)
w = alice_bob_scoring(counts)
print(f'\nGame advantage: {w:.4f}')

        ┌───┐     ┌───────┐ ░ ┌─┐   
   q_0: ┤ H ├──■──┤ Ry(0) ├─░─┤M├───
        └───┘┌─┴─┐├───────┤ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(0) ├─░──╫─┤M├
             └───┘└───────┘ ░  ║ └╥┘
meas: 2/═══════════════════════╩══╩═
                               0  1 
{'00': 545, '11': 479}
        ┌───┐     ┌──────────┐ ░ ┌─┐   
   q_0: ┤ H ├──■──┤ Ry(π/12) ├─░─┤M├───
        └───┘┌─┴─┐├──────────┤ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(π/12) ├─░──╫─┤M├
             └───┘└──────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════╩══╩═
                                  0  1 
{'00': 466, '11': 558}
        ┌───┐     ┌─────────┐ ░ ┌─┐   
   q_0: ┤ H ├──■──┤ Ry(π/6) ├─░─┤M├───
        └───┘┌─┴─┐├─────────┤ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(π/6) ├─░──╫─┤M├
             └───┘└─────────┘ ░  ║ └╥┘
meas: 2/═════════════════════════╩══╩═
                                 0  1 
{'00': 508, '11': 516}
        ┌───┐     ┌─────────┐ ░ ┌─┐   
   q_0: ┤ H ├──■──┤ Ry(π/4) ├─░─┤M├───
        └───┘┌─┴─┐├─────────┤ ░ └╥┘┌─┐
  

In [6]:
def alice_bob_circuit(a,b):
    qc = QuantumCircuit(2,2)
    qc.h(0)
    qc.cx(0,1)
    qc.measure([1],[0])
    if a==1:
        qc.h(0)
    if b==0:
        qc.ry(-math.pi/4.0,1)
    else:
        qc.ry(math.pi/4.0,1)
    qc.measure([0,1],[0,1])
    return qc

simulator = Aer.get_backend('aer_simulator')
counts = alice_bob_measurements(simulator)
w = alice_bob_scoring(counts)
print(f'\nGame advantage: {w:.4f}')


Alice: 0, Bob: 0
     ┌───┐                    ┌─┐   
q_0: ┤ H ├──■─────────────────┤M├───
     └───┘┌─┴─┐┌─┐┌──────────┐└╥┘┌─┐
q_1: ─────┤ X ├┤M├┤ Ry(-π/4) ├─╫─┤M├
          └───┘└╥┘└──────────┘ ║ └╥┘
c: 2/═══════════╩══════════════╩══╩═
                0              0  1 
{'00': 464, '11': 397, '10': 91, '01': 72}

Alice: 0, Bob: 1
     ┌───┐                   ┌─┐   
q_0: ┤ H ├──■────────────────┤M├───
     └───┘┌─┴─┐┌─┐┌─────────┐└╥┘┌─┐
q_1: ─────┤ X ├┤M├┤ Ry(π/4) ├─╫─┤M├
          └───┘└╥┘└─────────┘ ║ └╥┘
c: 2/═══════════╩═════════════╩══╩═
                0             0  1 
{'00': 447, '11': 442, '10': 67, '01': 68}

Alice: 1, Bob: 0
     ┌───┐     ┌───┐            ┌─┐   
q_0: ┤ H ├──■──┤ H ├────────────┤M├───
     └───┘┌─┴─┐└┬─┬┘┌──────────┐└╥┘┌─┐
q_1: ─────┤ X ├─┤M├─┤ Ry(-π/4) ├─╫─┤M├
          └───┘ └╥┘ └──────────┘ ║ └╥┘
c: 2/════════════╩═══════════════╩══╩═
                 0               0  1 
{'00': 261, '11': 244, '01': 248, '10': 271}

Alice: 1, Bob: 1
     ┌───┐   

In [10]:
def alice_bob_circuit(a,b):
    qc = QuantumCircuit(2,2)
    qc.h(0)
    qc.cx(0,1)
    if a==1:
        qc.h(0)
    if b==0:
        qc.ry((-math.pi/4.0)/divider,1)
    else:
        qc.ry((math.pi/4.0)/divider,1)
    qc.measure([0,1],[0,1])
    return qc


simulator = Aer.get_backend('aer_simulator')

for divider in range(1,10):
    counts = alice_bob_measurements(simulator,verbose=False)
    print(f'\nDivider: {divider}')
    w = alice_bob_scoring(counts)
    print(f'\nGame advantage: {w:.4f}')


Divider: 1

Alice: 0, Bob: 0
	RG: 79
	GR: 76
	RR: 456
	GG: 413
	win: 869, lose: 155
	advantage: 0.8486

Alice: 0, Bob: 1
	RG: 69
	GG: 444
	GR: 71
	RR: 440
	win: 884, lose: 140
	advantage: 0.8633

Alice: 1, Bob: 0
	RR: 455
	GG: 444
	GR: 61
	RG: 64
	win: 899, lose: 125
	advantage: 0.8779

Alice: 1, Bob: 1
	RR: 64
	RG: 413
	GG: 75
	GR: 472
	win: 885, lose: 139
	advantage: 0.8643

Game advantage: 0.8635

Divider: 2

Alice: 0, Bob: 0
	RG: 26
	GR: 23
	GG: 483
	RR: 492
	win: 975, lose: 49
	advantage: 0.9521

Alice: 0, Bob: 1
	GG: 483
	RR: 500
	RG: 23
	GR: 18
	win: 983, lose: 41
	advantage: 0.9600

Alice: 1, Bob: 0
	RR: 374
	RG: 165
	GG: 324
	GR: 161
	win: 698, lose: 326
	advantage: 0.6816

Alice: 1, Bob: 1
	GR: 342
	RR: 153
	GG: 171
	RG: 358
	win: 700, lose: 324
	advantage: 0.6836

Game advantage: 0.8193

Divider: 3

Alice: 0, Bob: 0
	GG: 530
	RR: 480
	GR: 7
	RG: 7
	win: 1010, lose: 14
	advantage: 0.9863

Alice: 0, Bob: 1
	RR: 503
	GG: 493
	RG: 14
	GR: 14
	win: 996, lose: 28
	advantage: 0.97